This is where the magic happens:

# MODELLING

In this notebook I'll work on a model to what a tourist will spend when vacationing in Tanzania.
The evaluation metric for the model is **Mean Absolute Error**.


To do:
- check if variables used in model are correct
- add comments to so far modelling steps
- feature selection/feature engineering
- check again outlier handling
- hyperparameter tuning
- fix overfitting
- error analysis
- interpretation/visualization

In [1]:
# import some packages that I'll need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# set color scheme
cpal = ["#f94144","#f3722c","#f8961e","#f9844a","#f9c74f","#90be6d","#43aa8b","#4d908e","#577590","#277da1"]

# seaborn theme
sns.set()

# use natural numbers
pd.options.display.float_format = "{:.2f}".format

# set random seed
RSEED = 42

In [2]:
# load data
TZA = pd.read_csv('data/Train.csv')

# load subregions
subregions = pd.read_csv('data/subregions.csv')
subregions.drop(['Unnamed: 0'], axis=1, inplace=True)

## Train Test Split

I'm going to split the train and test data now, very in the beginning to avoid data leakage.

I'm not using the sklearn train test split as I had concerning results in the first run (much better performance on test than on train data, I assume the different values of the target variable weren't well splitted). That's why I use Verstack stratified continuos split which allows me to stratify by continuos target variable. (It makes sure that different bins of total_cost are evenly divided among train and test data).

In [3]:
# train test split
train, test = scsplit(TZA, stratify = TZA['total_cost'], test_size = 0.3, random_state = RSEED)

## Preprocessing

I am going to preprocess the data now. I'll do it separately for train and test data. I start with the very basics for my Baseline Model.

#### Missing values and minor adjustments

In [4]:
# function to handle missing data and to make some basic adjustments on the dataset

def basic_preprocessing_baseline(df):
    # fill NaN total_male/total_female with 0
    df['total_male'] = df['total_male'].fillna(0)
    df['total_female'] = df['total_female'].fillna(0)
    
    # fill NaN travel_with with "Alone" if total_male plus total_female is one
    df.loc[df['total_female'] + df['total_male'] == 1, 'travel_with'] = 'Alone'
    
    # fill remaining NaN travel_with with missing
    df['travel_with'] = df['travel_with'].fillna('missing')
    
    # fill NaN most_impressing with "No comments"
    df['most_impressing'] = df['most_impressing'].fillna('No comments')
   
    # drop id column
    df = df.drop(['ID'], axis =1)
    
    return df

In [5]:
# apply function on train data
train_bl = basic_preprocessing_baseline(train)
# apply function on test data
test_bl = basic_preprocessing_baseline(test)

In [6]:
# separate target variable, both in train and test data

X_train_bl = train_bl.drop(['total_cost'], axis=1)
y_train_bl = train_bl['total_cost']

X_test_bl = test_bl.drop(['total_cost'], axis=1)
y_test_bl = test_bl['total_cost']

### Build Pipelines

I'm going to build some pipelines now. They'll make modelling easier and faster.

I start with a pipeline for the categorical features. I use a One Hot Encoder to convert them into numbers.

In [7]:
# create list of categorical features
cat_features_bl = list(X_train_bl.columns[X_train_bl.dtypes==object])

# build pipeline
cat_pipeline_bl = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown= 'ignore', drop = 'first'))
])

For the numerical features I'll use a Robust Scaler. It can handle outliers pretty good.

In [8]:
# create list of numerical features
num_features_bl = list(X_train_bl.columns[X_train_bl.dtypes!=object])

# build pipeline
num_pipeline_bl = Pipeline([
    ('rob_scaler', RobustScaler())
])

In [9]:
# combine both pipelines in a preprocessor
preprocessor_bl = ColumnTransformer([
    ('num', num_pipeline_bl, num_features_bl),
    ('cat', cat_pipeline_bl, cat_features_bl)
])

### Baseline Model

First I'm going to train a linear regression model. Except for some NaN imputations and the basic preprocessing we haven't made adjustments on the data yet. The result of the Baseline Model will serve me as a benchmark.

In [10]:
# build pipeline that combines the preprocessor and the linear regression model
pipe_linreg_bl = Pipeline([
    ('preprocessor', preprocessor_bl),
    ('linreg', LinearRegression())
])

In [11]:
# cross validate to check how the model performs on the train data
y_train_predicted_bl_cv = cross_val_predict(pipe_linreg_bl, X_train_bl, y_train_bl, cv=100)

# print MAE of Baseline Model (train data)
print("Mean Absolute Error Baseline Model (train data): {:.2f}".format(mean_absolute_error(y_train_bl, y_train_predicted_bl_cv)))

Mean Absolute Error Baseline Model (train data): 5858302.30


In [12]:
# fit the actual model
y_train_predicted_bl = pipe_linreg_bl.fit(X_train_bl, y_train_bl)

# make predictions for the test data
y_test_predicted_bl = pipe_linreg_bl.predict(X_test_bl)
# print MAE of Baseline Model (test data)
print("Mean Absolute Error Baseline Model (test data): {:.2f}".format(mean_absolute_error(y_test_bl, y_test_predicted_bl)))

Mean Absolute Error Baseline Model (test data): 5895242.84


#### Interpretation

The Mean Absolute Error of the Baseline Model is 5895242.84 Tanzanian Schillig TZS.
What does that mean?
The MAE is the sum of absolute errors divided by the sample size $n$ where $y_i$ is the prediction and $x_i$ is the true value:

$$ 
MAE = \frac {\sum_{i=1}^n \vert y_i - x_i \vert} {n}
$$

The MAE uses the same scale as the data, so in this case TZS. 

So, on average, the model's predictions are 5895242.84 TZS off the true value. This is roughly 2156 Euro and seems quite a lot. 

Our aim is to improve (lower) this metric as much as we can.

So let's start with the actual 

### Modelling

First I'll make some more adjustments on the dataset - some of them we already saw in the EDA. I want to reduce noise from the dataset and add some more information which seems useful to me. I need a function that I can apply to both the train and the test data.

In [13]:
# function to handle missing data and to make some adjustments in the data set

def adjustments1(df):
    # fill NaN total_male/total_female with 0
    df['total_male'] = df['total_male'].fillna(0)
    df['total_female'] = df['total_female'].fillna(0)
    
    # add a column group_size based on total_male/total_female
    df['group_size'] = df['total_female'] + df['total_male']
    
    # fill NaN travel_with with "Alone" if group_size is one
    df.loc[df.group_size == 1, 'travel_with'] = 'Alone'
    
    # fill remaining NaN travel_with with missing
    df['travel_with'] = df['travel_with'].fillna('missing')
    
    # fill NaN most_impressing with "No comments"
    df['most_impressing'] = df['most_impressing'].fillna('No comments')
    
    # add a column total_nights based on night_zanzibar/night_mainland
    df['total_nights'] = df['night_zanzibar'] + df['night_mainland']
    
    # handle group_size equals zero: either replace by 1 if alone traveller or median group size of the train data
    df.loc[(df.group_size == 0) & (df.travel_with == 'Alone'), 'group_size'] = 1
    df.loc[df.group_size == 0, 'group_size'] = train['group_size'].median()

    # handle total_nights equals zero: replace by median total_nights of the train data
    df.loc[df.total_nights == 0, 'total_nights'] = train['total_nights'].median()

    # drop id column
    df = df.drop(['ID'], axis =1)
    
    # drop night_mainland column (to avoid multicollinearity)
    df = df.drop(['night_mainland'], axis =1)
    
    # drop total_male column (to avoid multicollinearity)
    df = df.drop(['total_male'], axis =1)

    # add subregions just as in the EDA
    df['country'] = df['country'].str.lower()
    df = df.replace({'country' : {'united states of america': 'united States', 'swaziland' : 'eswatini', 'cape verde' : 'cabo verde', 'swizerland' : 'switzerland', 'ukrain' : 'ukraine','malt' : 'malta', 'burgaria' : 'bulgaria', 'korea' : 'south korea', 'comoro' : 'comoros', 'scotland' : 'united kingdom', 'russia' : 'russia', 'srilanka': 'sri lanka'}})
    df = df.replace({'country' : {'ivory coast': "côte d'ivoire", 'drc' : 'congo', 'uae' : 'united arab emirates', 'trinidad tobacco' : 'trinidad and tobago', 'costarica' : 'costa rica', 'philipines' : 'philippines', 'djibout' : 'djibouti', 'morroco' : 'morocco'}})
    df['country'] = df['country'].str.capitalize()
    df = pd.merge(df, subregions, how ='left')
    
    return df

In [14]:
# apply to train and test data
train_model = adjustments1(train)
test_model = adjustments1(test)

We have some outliers in the numeric columns, including our target variable. I want to get rid of them and try (for the first time) an outlier detection algorithm:

### Outlier handling: Isolation Forest

In this first iteration I'll use the Isolation Forest to detect anomalies and I'll then delete these observations. (Possibly I'll handle this differently in another iteration.) It is important that we apply this method to the train data only. Outliers in the test data remain untouched.
The Isolation Forest will not instantly delete the columns, but give a score for each row: either -1 (anomaly) or 1 (normal)

In [15]:
# create a dataframe with only numeric columns
iso_features = train_model[['total_female', 'night_zanzibar', 'total_cost', 'group_size',
       'total_nights']]

# have a look
iso_features

,total_female,night_zanzibar,total_cost,group_size,total_nights
0,1.00,0.00,4233255.00,1.00,3.00
1,3.00,4.00,52377000.00,5.00,10.00
2,0.00,4.00,7458750.00,3.00,15.00
3,2.00,3.00,828750.00,3.00,3.00
4,1.00,0.00,2836662.50,2.00,7.00
...,...,...,...,...,...
3361,2.00,0.00,16112243.00,4.00,6.00
3362,1.00,0.00,3987945.00,1.00,12.00
3363,0.00,0.00,314925.00,1.00,3.00
3364,0.00,0.00,1560000.00,1.00,4.00


In [16]:
# check size of this data set
train_model.shape

(3366, 23)

In [17]:
# define isolation forest
isolation_forest = IsolationForest()

# fit isolation forest
isolation_forest.fit(iso_features)

# make predictions
predictions = isolation_forest.predict(iso_features)

# create data frame with Isolation Forest Scores
predictions_df = pd.DataFrame(predictions, columns = ['Iso_Score'])

# merge this data frame into the train_model data frame
train_model = pd.concat([train_model, predictions_df], axis=1)

# keep only those with Iso_Score is 1 (normal)
train_model = train_model[train_model['Iso_Score'] == 1]

# delete the Iso_Score column
train_model = train_model.drop(['Iso_Score'], axis =1)

In [18]:
# check size of data set without anomalies
train_model.shape

(2936, 23)

430 observations were deleted. We might find a better way for the outlier handling later.

I'm now going to use a 

### Random Forest Regressor

In [21]:
# define target variable

X_train = train_model.drop(['total_cost'], axis=1)
y_train = train_model['total_cost']

X_test = test_model.drop(['total_cost'], axis=1)
y_test = test_model['total_cost']

In [22]:
# build pipeline

cat_features = list(X_train.columns[X_train.dtypes==object])

cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown= 'ignore', drop = 'first'))
])

num_features = list(X_train.columns[X_train.dtypes!=object])

num_pipeline = Pipeline([
    ('rob_scaler', RobustScaler())
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features),
])

pipe_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('randfor', RandomForestRegressor(random_state=RSEED))
])

In [23]:
# cross validate

y_train_predicted_cv = cross_val_predict(pipe_rf, X_train, y_train, cv=5)
print("Mean Absolute Error first Model (train data): {:.2f}".format(mean_absolute_error(y_train, y_train_predicted_cv)))

Mean Absolute Error first Model (train data): 3523182.54


In [24]:
# fit and predict

y_train_predicted = pipe_rf.fit(X_train, y_train)

y_test_predicted = pipe_rf.predict(X_test)
print("Mean Absolute Error first Model (test data): {:.2f}".format(mean_absolute_error(y_test, y_test_predicted)))

Mean Absolute Error first Model (test data): 5031151.30
